In [1]:
import pandas as pd
import os
from functools import reduce
#import pickle
#import HDF5

In [2]:
TADs = pd.read_csv('merge_TADs.bed',sep='\t')
#TADs_strength = pd.read_csv('TADs_minmaxStrength.tsv',sep='\t')

In [3]:
test_dfs = []
ChIP_files = [f for f in os.listdir(os.getcwd()) if f.endswith('.bedgraph')]
ChIP_names = [f.strip('.bedgraph') for f in ChIP_files]
for file_ in [f for f in os.listdir(os.getcwd()) if f.endswith('.bedgraph')]:
    df = pd.read_csv(file_,index_col=None,header=None,sep='\t')
    df.columns = ['chrom','start','end']+[file_.strip('.bedgraph')]
    test_dfs.append(df)

# concatenate all dfs into one
test_ChIPs = reduce(lambda  left,right: pd.merge(left,right,on=['chrom','start','end'],
                                             how='outer'), test_dfs)
#test_ChIPs.columns = ['chrom','start','end']+ChIP_names

In [4]:
test_ChIPs = test_ChIPs.fillna(0)

In [5]:
# def get_loc(ChIP_start,ChIP_end,TAD_start,TAD_end):
#     up = (TAD_start<=ChIPs_chr.start) & (ChIPs_chr.end<=TAD_start+40000)
#     down = (TAD_end<=ChIPs_chr.start) & (ChIPs_chr.end<=TAD_end+40000)
#     return up|down
chroms = ['chr'+str(i) for i in range(1,20)]+['chrX']
#chroms = ['chr'+str(i) for i in range(1,3)]
TAD_ChIP_chrs = []
for chrom in chroms:
    TADs_chr = TADs.loc[TADs.chrom==chrom].reset_index().drop(['index'],axis=1)
    ChIPs_chr = test_ChIPs.loc[test_ChIPs.chrom==chrom].reset_index().drop(['index'],axis=1)
    ChIP_TAD_means = []
    TADs_chr = TADs_chr    
    for idx,(c,s,e) in TADs_chr.iterrows():
        #print((s<=ChIPs_chr.start) & (ChIPs_chr.start<=s+40000))
        ChIP_TAD_up = ChIPs_chr.loc[(s<=ChIPs_chr.start) & (ChIPs_chr.end<=s+40000),[False]*3+[True]*8]
        ChIP_TAD_down = ChIPs_chr.loc[(e<=ChIPs_chr.start) & (ChIPs_chr.end<=e+40000),[False]*3+[True]*8]
        #print(ChIPs_chr.iloc[:,[True]+[False]*1,0])
        ChIP_TAD_up_mean = ChIP_TAD_up.mean(axis=0)
        ChIP_TAD_down_mean = ChIP_TAD_down.mean(axis=0)
        ChIP_TAD_value = pd.DataFrame([ChIP_TAD_up_mean.combine(ChIP_TAD_down_mean, max, 0)])
        ChIP_TAD_means.append(ChIP_TAD_value)
    tmp = pd.concat(ChIP_TAD_means,ignore_index=True,axis=0)
    #tmp_chr = pd.concat([TADs_chr,tmp],axis=1)
    tmp_chr = TADs_chr.join(tmp,how='inner')
    TAD_ChIP_chrs.append(tmp_chr)
TAD_ChIP = pd.concat(TAD_ChIP_chrs,ignore_index=True,axis=0)
TAD_ChIP.columns = ['chrom','start','end']+ChIP_names
#TAD_ChIP['strength'] = TADs_strength.iloc[:,3]
#TAD_ChIP = TAD_ChIP.fillna(0)

In [6]:
TAD_ChIP.to_csv('merge_TAD_ChIP.tsv',sep='\t',index=False,quoting=3)
#store = pd.HDFStore('store.h5')
#store['TAD_ChIP']=TAD_ChIP
    

In [8]:
TAD_ChIP.head()

,chrom,start,end,H3K9me3_RPKM,H3K36me3_RPKM,H3K4me1_RPKM,POU5F1_RPKM,H3K27me3_RPKM,NANOG_RPKM,CTCF_RPKM,H3K4me3_RPKM,strength
0,chr1,4480000.0,4800000.0,0.672959,3.081767,3.183183,0.780747,3.278561,0.993484,0.880504,2.843474,0.243850
1,chr1,4840000.0,6200000.0,0.233456,3.438194,2.542468,0.842724,0.222650,1.005763,0.841192,2.932617,0.978933
2,chr1,6240000.0,9520000.0,0.435774,1.520725,1.213257,0.618981,0.360319,0.708752,0.778033,2.703058,0.351045
3,chr1,9560000.0,10160000.0,1.104924,1.509293,0.463018,0.717030,0.622939,0.785659,0.774021,0.454601,0.961099
4,chr1,10200000.0,11600000.0,0.646473,1.251942,0.622932,0.676496,0.860671,0.715203,0.706546,1.531882,0.395564
